# Load and Clean the Data

In [15]:
%run ./preprocessing.ipynb

E    52268
K    13372
Name: customer_gender, dtype: int64


In [16]:
trainset_male, valset_male, evalset_male = dataset_splits(df_male)
trainset_male.shape, valset_male.shape, evalset_male.shape

((36587, 42), (5227, 42), (10454, 42))

In [17]:
trainset_female, valset_female, evalset_female = dataset_splits(df_female)
trainset_female.shape, valset_female.shape, evalset_female.shape

((9360, 42), (1337, 42), (2675, 42))

# Self-Supervised Similarity Measure

In [19]:
%run ./matching_algorithm.ipynb

cannot import name 'linear_sum_assignment' from 'scipy.sparse.csgraph' (/home/mmaddox/.local/lib/python3.8/site-packages/scipy/sparse/csgraph/__init__.py)
name 'linear_sum_assignment' is not defined
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/mmaddox/.local/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/1840fb884be34b399af112ee256325d5-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/1840fb884be34b399af112ee256325d5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10005 COLUMNS
At line 30206 RHS
At line 40207 BOUNDS
At line 40408 ENDATA
Problem MODEL has 10000 rows, 200 columns and 20000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10000 (0) rows, 200 (0) columns and 20000 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 0
0  Obj -0 Dual inf 1.999